## Feature correlation analysis

## Context

with 400 + features, manu derived from similar signals, redudancy is inevitable. Highly correlated features add noise, slow down training, and can destabilize linear model. this analysis identifies clusters of correlated features for pruning

## Objective 
- compute correaltion metrices for numeric features
- identify highly correlated feature paris(|r| > 0.95)
- select candidates for removal based on correalatin and importance
- quantify potential dimensionality reduction


In [1]:
# import  library and load data

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings

warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8-whitegrid')

train = pd.read_parquet(Path('../data/interim/train_merged.parquet'))
print(f'Data loaded: {train.shape}')

Data loaded: (590540, 434)


In [3]:
# numeric features
numeric_cols = train.select_dtypes(include=[np.number]).columns.tolist()
numeric_cols = [c for c in numeric_cols if c not in ['Transactionid', 'isFraud']]

print(f'Numeric features : {len(numeric_cols)}')

# sample for speed
sample = train[numeric_cols].sample(n=50000, random_state=42)

Numeric features : 402
